In [1]:
from itertools import chain

import numpy as np
import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer,classification_report
from sklearn.model_selection import cross_val_score, RandomizedSearchCV, train_test_split, cross_val_predict

import sklearn_crfsuite
from sklearn_crfsuite import scorers, CRF, metrics

In [2]:
import pandas as pd

data = pd.read_csv('ner_dataset.csv', encoding='latin1', low_memory=False, dtype={'Sentence #': str, 'Word': str, 'POS': str, 'Tag': str})
filled_data = data.fillna(method='ffill')

class SentenceGetter(object):
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                            s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
  
    def get_next(self):     
        try:
             s = self.grouped["Sentence: {}".format(self.n_sent)]
             self.n_sent += 1
             return s
        except:
             return None       

getter = SentenceGetter(filled_data)
sentences = getter.sentences

In [3]:
%%time
# train_sents = list(nltk.corpus.conll2002.iob_sents('esp.train'))
# test_sents = list(nltk.corpus.conll2002.iob_sents('esp.testb'))

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

# X = [sent2features(s) for s in train_sents]
# y = [sent2labels(s) for s in train_sents]

# X_test = [sent2features(s) for s in test_sents]
# y_test = [sent2labels(s) for s in test_sents]

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print(X_test[7])

[{'bias': 1.0, 'word.lower()': 'there', 'word[-3:]': 'ere', 'word[-2:]': 're', 'word.isupper()': False, 'word.istitle()': True, 'postag': 'EX', 'postag[:2]': 'EX', 'BOS': True, '+1:word.lower()': 'is', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'VBZ', '+1:postag[:2]': 'VB'}, {'bias': 1.0, 'word.lower()': 'is', 'word[-3:]': 'is', 'word[-2:]': 'is', 'word.isupper()': False, 'word.istitle()': False, 'postag': 'VBZ', 'postag[:2]': 'VB', '-1:word.lower()': 'there', '-1:word.istitle()': True, '-1:word.isupper()': False, '-1:postag': 'EX', '-1:postag[:2]': 'EX', '+1:word.lower()': 'no', '+1:word.istitle()': False, '+1:word.isupper()': False, '+1:postag': 'DT', '+1:postag[:2]': 'DT'}, {'bias': 1.0, 'word.lower()': 'no', 'word[-3:]': 'no', 'word[-2:]': 'no', 'word.isupper()': False, 'word.istitle()': False, 'postag': 'DT', 'postag[:2]': 'DT', '-1:word.lower()': 'is', '-1:word.istitle()': False, '-1:word.isupper()': False, '-1:postag': 'VBZ', '-1:postag[:2]': 'VB', '+1:


Wall time: 2.42 s


In [4]:
%%time
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=False
)
crf.fit(X_train, y_train)

# pred = cross_val_predict(estimator=crf, X=X, y=y, cv=5, n_jobs=-1, verbose=1)
# report = classification_report(y_pred=pred, y_true=y)
# print(report)

labels = list(crf.classes_)
labels.remove('O')

Wall time: 2min 10s


In [ ]:
print(labels)

['B-PER', 'I-PER', 'B-TIM', 'B-MAL', 'B-ORG', 'I-ORG', 'B-GPE', 'I-MAL', 'I-TIM', 'B-NAT', 'I-GPE', 'B-EVE', 'B-ART', 'I-EVE', 'I-ART', 'I-NAT']


In [ ]:
%%time
# define fixed parameters and parameters to search
crf = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=False
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=4,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [ ]:
import pickle
filename = 'rs_finalized_model.sav'
pickle.dump(rs, open(filename, 'wb'))

In [ ]:
print('best params:', rs.best_params_)
print('best cv score:', rs.best_score_)
print('model size {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

In [ ]:
crf = rs.best_estimator_
y_pred = crf.predict(X_test)

sorted_labels = sorted(
    labels,
    key=lambda name: (name[1:], name[0])
)

print(metrics.flat_classification_report(
    y_test, y_pred, labels=sorted_labels, digits=3
))

In [ ]:
from collections import Counter

def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f"%(label_from, label_to, weight))

print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))

print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])


In [ ]:
import eli5

eli5.show_weights(crf, top=30)